In [41]:
import gurobipy as gp
from gurobipy import Model, GRB
import pandas as pd
import numpy as np

In [204]:
path = r'C:\Users\anant\OneDrive\Desktop\MBAN\OMIS 6000\Assignment\2'
file = '\price_response.csv'

file_path = path + file

In [219]:
price_df = pd.read_csv(file_path) 

In [206]:
price_df

,Product,Intercept,Sensitivity,Capacity
0,Line 1 Product 1,35234.545786,-45.896450,80020.0
1,Line 1 Product 2,37790.240832,-8.227794,89666.0
2,Line 1 Product 3,35675.333217,-7.584436,80638.0
3,Line 2 Product 1,37041.380378,-9.033166,86740.0
4,Line 2 Product 2,36846.140386,-4.427869,84050.0
5,Line 2 Product 3,35827.023747,-2.629060,86565.0
6,Line 3 Product 1,39414.266325,-2.421484,87051.0
7,Line 3 Product 2,35991.951460,-4.000512,85156.0
8,Line 3 Product 3,39313.317031,-2.296622,87588.0


In [224]:
# Basic version coefficients

a1, capacity1 = price_df.loc[0, ['Intercept', 'Capacity']]

In [222]:
b1 = -price_df.loc[0, 'Sensitivity']

In [227]:
# Advanced version coefficients

a2, capacity2 = price_df.loc[1, ['Intercept', 'Capacity']]

In [228]:
b2 = -price_df.loc[1, 'Sensitivity']

In [229]:
m = Model("Optimal_Pricing")

In [230]:
# Add variables for prices

p1 = m.addVar(lb=0, name="Price_Basic")
p2 = m.addVar(lb=0, name="Price_Advanced")

In [231]:
# Objective: Maximize total revenue considering the linear price response functions

m.setObjective(p1 * (a1 - b1 * p1) + p2 * (a2 - b2 * p2), GRB.MAXIMIZE)

In [232]:
# Constraint: The Advanced version should have a higher price compared to the Basic one

m.addConstr(p2 >= p1, "Price_Ordering")

<gurobi.Constr *Awaiting Model Update*>

In [233]:
# Add capacity constraints

m.addConstr(a1 - b1 * p1 <= capacity1, "Capacity_Basic")

<gurobi.Constr *Awaiting Model Update*>

In [234]:
m.addConstr(a2 - b2 * p2 <= capacity2, "Capacity_Advanced")

<gurobi.Constr *Awaiting Model Update*>

In [235]:
# Optimize the model

m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 2 columns and 4 nonzeros
Model fingerprint: 0x86196f40
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [2e+01, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 5e+04]
Presolve removed 2 rows and 0 columns
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     

In [236]:
print(f"Optimal Price for Basic Version: {p1.X}")

Optimal Price for Basic Version: 383.84827160870276


In [237]:
print(f"Optimal Price for Advanced Version: {p2.X}")

Optimal Price for Advanced Version: 2296.498918132063


In [238]:
print(f"Maximized Revenue: {m.ObjVal}")

Maximized Revenue: 50154983.34381363


2.

In [239]:
# Initialize prices

p1, p2 = 0, 0

In [240]:
# Step size

step_size = 0.001

In [241]:
# Stopping criterion

tol = 1e-6

In [242]:
delta = 1

In [243]:
while delta > tol:
   
    # Calculate gradients based on the revenue function's derivative
    grad_p1 = a1 - 2 * b1 * p1
    grad_p2 = a2 - 2 * b2 * p2

    # Update prices based on gradients
    new_p1 = p1 + step_size * grad_p1
    new_p2 = p2 + step_size * grad_p2

    # Quadratic programming model for projection
    
    gd_m = Model("gd_Optimal_Pricing")
    proj_p1 = gd_m.addVar(lb=0, ub=capacity1, name="proj_p1")
    proj_p2 = gd_m.addVar(lb=0, ub=capacity2, name="proj_p2")

    # Objective: Minimize the distance between new prices and projected prices
    gd_m.setObjective((proj_p1 - new_p1)*(proj_p1 - new_p1) + (proj_p2 - new_p2)*(proj_p2 - new_p2), GRB.MINIMIZE)

    # Constraints
    gd_m.addConstr(proj_p2 >= proj_p1, "PriceOrdering")

    # Optimize the model
    gd_m.optimize()

    # Extract the projected prices
    updated_p1 = proj_p1.X
    updated_p2 = proj_p2.X

    # Calculate the maximum change in price for the stopping criterion
    delta = max(abs(updated_p1 - p1), abs(updated_p2 - p2))

    # Update the prices for the next iteration
    p1, p2 = updated_p1, updated_p2

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 2 columns and 2 nonzeros
Model fingerprint: 0x4970b0ba
Model has 2 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+01, 8e+01]
  QObjective range [2e+00, 2e+00]
  Bounds range     [8e+04, 9e+04]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.01s
Presolved: 1 rows, 2 columns, 2 nonzeros
Presolved model has 2 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 0.000e+00
 Factor NZ  : 1.000e+00
 Factor Ops : 1.000e+00 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   7.09070599e+09 -

In [244]:
print(f"Optimal Price for Basic Version: {p1}")
print(f"Optimal Price for Advanced Version: {p2}")

Optimal Price for Basic Version: 383.8482716266063
Optimal Price for Advanced Version: 2296.4988588233246


3.

In [266]:
quad_m = Model("quadratic_optimization")

In [267]:
prices = quad_m.addVars(price_df.index, lb=0, name="p")

In [268]:
prices

{0: <gurobi.Var *Awaiting Model Update*>,
 1: <gurobi.Var *Awaiting Model Update*>,
 2: <gurobi.Var *Awaiting Model Update*>,
 3: <gurobi.Var *Awaiting Model Update*>,
 4: <gurobi.Var *Awaiting Model Update*>,
 5: <gurobi.Var *Awaiting Model Update*>,
 6: <gurobi.Var *Awaiting Model Update*>,
 7: <gurobi.Var *Awaiting Model Update*>,
 8: <gurobi.Var *Awaiting Model Update*>}

In [269]:
# Objective: Maximize total revenue considering the price-response function for each product

quad_m.setObjective(sum(prices[i] * (price_df.loc[i, 'Intercept'] + price_df.loc[i, 'Sensitivity'] * prices[i]) for i in price_df.index), GRB.MAXIMIZE)

In [270]:
# Constraints: Price ordering within each product line

for line in range(1, 4):  # Assuming 3 lines
    for version in range(1, 3):  # Basic to Advanced, Advanced to Premium
        quad_m.addConstr(prices[(line-1)*3 + version - 1] <= prices[(line-1)*3 + version], name=f"ordering_{line}_{version}")

In [271]:
# Adding capacity constraints

for i in price_df.index:
    demand = price_df.loc[i, 'Intercept'] + price_df.loc[i, 'Sensitivity'] * prices[i]
    quad_m.addConstr(demand <= price_df.loc[i, 'Capacity'], name=f"capacity_{i}")


In [272]:
# Optimize the model

quad_m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 15 rows, 9 columns and 21 nonzeros
Model fingerprint: 0xeb9d26fa
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 5e+04]
Presolve removed 9 rows and 0 columns
Presolve time: 0.00s
Presolved: 6 rows, 9 columns, 12 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 3.000e+00
 Factor NZ  : 9.000e+00
 Factor Ops : 1.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual  

In [273]:
for constr in quad_m.getConstrs():
    print(f"Name: {constr.ConstrName}, Expression: {quad_m.getRow(constr)}")

Name: ordering_1_1, Expression: p[0] + -1.0 p[1]
Name: ordering_1_2, Expression: p[1] + -1.0 p[2]
Name: ordering_2_1, Expression: p[3] + -1.0 p[4]
Name: ordering_2_2, Expression: p[4] + -1.0 p[5]
Name: ordering_3_1, Expression: p[6] + -1.0 p[7]
Name: ordering_3_2, Expression: p[7] + -1.0 p[8]
Name: capacity_0, Expression: -45.89644970638436 p[0]
Name: capacity_1, Expression: -8.22779417263456 p[1]
Name: capacity_2, Expression: -7.584436409583301 p[2]
Name: capacity_3, Expression: -9.033166404486591 p[3]
Name: capacity_4, Expression: -4.4278692064433125 p[4]
Name: capacity_5, Expression: -2.6290600153591 p[5]
Name: capacity_6, Expression: -2.421483918369876 p[6]
Name: capacity_7, Expression: -4.000512400639974 p[7]
Name: capacity_8, Expression: -2.296622373087237 p[8]


In [274]:
# Extract optimal prices and calculate total revenue

optimal_prices = {v.VarName: v.X for v in quad_m.getVars()}
optimal_revenue = quad_m.ObjVal

In [275]:
optimal_prices

{'p[0]': 383.8482716083725,
 'p[1]': 2296.4989181049355,
 'p[2]': 2351.8776670489674,
 'p[3]': 2050.2987944234346,
 'p[4]': 4160.707856083369,
 'p[5]': 6813.656504083383,
 'p[6]': 5870.9328095980045,
 'p[7]': 5870.932809598009,
 'p[8]': 8558.942360734152}

In [276]:
optimal_revenue

718382097.6700904

4.

In [277]:
quad_m_b = Model("quadratic_optimization_all_constraints")

In [278]:
prices = quad_m_b.addVars(price_df.index, lb=0, name="p")

In [279]:
# Objective: Maximize total revenue considering the price-response function for each product

quad_m_b.setObjective(sum(prices[i] * (price_df.loc[i, 'Intercept'] + price_df.loc[i, 'Sensitivity'] * prices[i]) for i in price_df.index), GRB.MAXIMIZE)

In [280]:
# Constraints: Price ordering within each product line

for line in range(1, 4):  # Assuming 3 lines
    for version in range(1, 3):  # Basic to Advanced, Advanced to Premium
        quad_m_b.addConstr(prices[(line-1)*3 + version - 1] <= prices[(line-1)*3 + version], name=f"ordering_within_{line}_{version}")


In [281]:
# Adding capacity constraints

for i in price_df.index:
    demand = price_df.loc[i, 'Intercept'] + price_df.loc[i, 'Sensitivity'] * prices[i]
    quad_m_b.addConstr(demand <= price_df.loc[i, 'Capacity'], name=f"capacity_{i}")

In [282]:
# Constraints for price increases across product lines for the same version

for version in range(3):
    quad_m_b.addConstr(prices[version] <= prices[version + 3], name=f"cross_ordering_1_to_2_{version}")
    quad_m_b.addConstr(prices[version + 3] <= prices[version + 6], name=f"cross_ordering_2_to_3_{version}")

In [283]:
# Optimize the model

quad_m_b.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11.0 (22621.2))

CPU model: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 21 rows, 9 columns and 33 nonzeros
Model fingerprint: 0x748468e3
Model has 9 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [4e+04, 4e+04]
  QObjective range [5e+00, 9e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+04, 5e+04]
Presolve removed 9 rows and 0 columns
Presolve time: 0.01s
Presolved: 12 rows, 9 columns, 24 nonzeros
Presolved model has 9 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.200e+01
 Factor NZ  : 7.800e+01
 Factor Ops : 6.500e+02 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual 

In [284]:
for constr in quad_m_b.getConstrs():
    print(f"Name: {constr.ConstrName}, Expression: {quad_m_b.getRow(constr)}")

Name: ordering_within_1_1, Expression: p[0] + -1.0 p[1]
Name: ordering_within_1_2, Expression: p[1] + -1.0 p[2]
Name: ordering_within_2_1, Expression: p[3] + -1.0 p[4]
Name: ordering_within_2_2, Expression: p[4] + -1.0 p[5]
Name: ordering_within_3_1, Expression: p[6] + -1.0 p[7]
Name: ordering_within_3_2, Expression: p[7] + -1.0 p[8]
Name: capacity_0, Expression: -45.89644970638436 p[0]
Name: capacity_1, Expression: -8.22779417263456 p[1]
Name: capacity_2, Expression: -7.584436409583301 p[2]
Name: capacity_3, Expression: -9.033166404486591 p[3]
Name: capacity_4, Expression: -4.4278692064433125 p[4]
Name: capacity_5, Expression: -2.6290600153591 p[5]
Name: capacity_6, Expression: -2.421483918369876 p[6]
Name: capacity_7, Expression: -4.000512400639974 p[7]
Name: capacity_8, Expression: -2.296622373087237 p[8]
Name: cross_ordering_1_to_2_0, Expression: p[0] + -1.0 p[3]
Name: cross_ordering_2_to_3_0, Expression: p[3] + -1.0 p[6]
Name: cross_ordering_1_to_2_1, Expression: p[1] + -1.0 p[4]


In [285]:
# Extract optimal prices and calculate total revenue

optimal_prices = {v.VarName: v.X for v in quad_m_b.getVars()}
optimal_revenue = quad_m_b.ObjVal

In [286]:
optimal_prices

{'p[0]': 383.84827160837165,
 'p[1]': 2296.498918116899,
 'p[2]': 2351.877667035985,
 'p[3]': 2050.298794423435,
 'p[4]': 4160.707856083367,
 'p[5]': 6813.656504083378,
 'p[6]': 5870.932809598007,
 'p[7]': 5870.932809598009,
 'p[8]': 8558.942360734152}

In [287]:
optimal_revenue

718382097.6700902

5.

- Scenario (d) added constraints to ensure that prices increase not only within each product line but also across the different versions of the products (ensuring, for example, that a FusionBook Elite is more expensive than an InfiniteEdge Notebook, which in turn is more expensive than an EvoTech Pro).

- This approach provides a consistent pricing logic across the entire product range, potentially simplifying the overall pricing structure and making it easier for customers to navigate choices across different product lines.

-  It allows TechEssentials Inc. to clearly differentiate between its product lines, making it easier for consumers to understand the hierarchy and value proposition of each product.

- It aligns with customer expectations that a product with more features or from a supposedly higher-end line should cost more.

6.

- The model assumes a linear relationship between price and demand, which is a simplification. In reality, the relationship might be non-linear. For example, small price changes might not significantly affect demand until a certain threshold is reached, beyond which demand might drop sharply. Including non-linear effects could provide a more accurate representation of how price changes affect demand.

- The model uses a single sensitivity value for each product, which assumes uniform price sensitivity across all potential customers. However, different customer segments may have varying sensitivities to price changes. A model that differentiates between these segments could allow for more nuanced and effective pricing strategies.